In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jun  5 14:33:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
pip install -U transformers memory_profiler datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 12.0 MB/s 
     |████████████████████████████████| 346 kB 59.6 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 4.0 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 140 kB 49.6 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 212 kB 64.1 MB/s 
     |████████████████████████████████| 1.1 MB 46.3 MB/s 
     |████████████████████████████████| 127 kB 43.8 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 45.7 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 
     |████████████████████████████████| 112 kB 61.1 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.60.0-py3-none-any.whl size=31284

In [3]:
import torch, gc, random, datasets
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
%load_ext memory_profiler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [4]:
def set_seed(seed: int):
  random.seed(seed)
  np.random.seed(seed)
  if is_torch_available():
      torch.manual_seed(seed)
      torch.cuda.manual_seed_all(seed)

  if is_tf_available():
      import tensorflow as tf
 
      tf.random.set_seed(seed)
 
set_seed(42)

In [5]:
df = pd.read_csv("/content/Womens Clothing E-Commerce Reviews.csv")

In [6]:
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [7]:
# print("Range:"+"("+str(df.total_score.min())+", "+str(df.total_score.max())+")")

In [8]:
# df['Review Text'].str.split(' ').str.len().max()

In [9]:
# df = df[~df.total_score.isna()]
# # df = df[~(df.score_1.isin(['0,0', '0,5', '1,0', '1,5', '2,0',]))]
# df = df[~(df.total_score.astype(float)==0)]

In [10]:
# len(df)

In [17]:
def LinearRegression_with_Transformer(model_name: str, Data: pd.Series, Target:pd.Series, test_size: np.float64, max_length: int, num_labels: int, num_epochs: int):

  # Make data
  X = Data
  y = Target


  # Split Data
  X_train, X_test, y_train, y_test = train_test_split(X.tolist(), y, test_size=test_size)

  # Call the Tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  # Encode the text
  train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
  valid_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=max_length)



  class MakeTorchData(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
          item["labels"] = torch.tensor([self.labels[idx]])
          item["labels"] = float(item["labels"])
          return item

      def __len__(self):
          return len(self.labels)

  # convert our tokenized data into a torch Dataset
  train_dataset = MakeTorchData(train_encodings, y_train.ravel())
  valid_dataset = MakeTorchData(valid_encodings, y_test.ravel())


  # Call Model
  model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels).to("cuda") # np.log(1000)

  # def inverse_sigmoid(logits):
  #   return -np.log((1.0 / (logits + 1e-8)) - 1.0)

  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import mean_absolute_error
  from sklearn.metrics import mean_squared_error
  from sklearn.metrics import r2_score

  def compute_metrics_for_regression(eval_pred):
      logits, labels = eval_pred
      # print("Logits:", logits[0:5])
      # print("Labels:", labels[0:5])
      # logits = inverse_sigmoid(logits)
      labels = labels.reshape(-1, 1)
      
      print("Logits:", logits[0:5])
      print("Labels:", labels[0:5])
      # print("Labels:", labels)

      mse = mean_squared_error(labels, logits)
      rmse = mean_squared_error(labels, logits, squared=False)
      mae = mean_absolute_error(labels, logits)
      r2 = r2_score(labels, logits)
      # smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
      single_squared_errors = ((logits - labels).flatten()**2).tolist()
      accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
      
      return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "accuracy": accuracy}

  # Specifiy the arguments for the trainer  
  training_args = TrainingArguments(
      output_dir='./results',          # output directory
      num_train_epochs=num_epochs,     # total number of training epochs
      per_device_train_batch_size=64,   # batch size per device during training
      per_device_eval_batch_size=20,   # batch size for evaluation
      weight_decay=0.01,               # strength of weight decay
      learning_rate=2e-5,
      # logging_dir='./logs',            # directory for storing logs
      save_total_limit=10,
      load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
      metric_for_best_model = 'accuracy',    # select the base metrics
      evaluation_strategy="epoch",
      save_strategy="epoch",
  ) 

  # Call the Trainer
  trainer = Trainer(
      model=model,                         # the instantiated Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=valid_dataset,          # evaluation dataset
      compute_metrics=compute_metrics_for_regression,     # the callback that computes metrics of interest
  )

  # Train the model
  trainer.train()
  
  # Call the summary
  trainer.evaluate()



  return trainer, model

In [12]:
%%time
%%memit
bert_trainer, bert_model = LinearRegression_with_Transformer(model_name = 'neuralmind/bert-large-portuguese-cased', 
                                                                 Data = df.essay_text.astype(str), 
                                                                 Target = df.total_score.astype(float), 
                                                                 test_size = 0.2, 
                                                                 max_length = 512, 
                                                                 num_labels = 1, 
                                                                 num_epochs = 100)

Downloading:   0%|          | 0.00/155 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from th

Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Smape
1,No log,0.192932,0.192932,0.439241,0.390277,-2.478255,13.924477
2,No log,0.049018,0.049018,0.221400,0.185570,0.116285,7.139155
3,No log,0.083286,0.083286,0.288593,0.219865,-0.501508,8.309401
4,No log,0.033807,0.033807,0.183866,0.134562,0.390520,5.235734
5,0.125500,0.040338,0.040338,0.200843,0.144808,0.272770,5.612128
6,0.125500,0.086158,0.086158,0.293528,0.240284,-0.553297,8.866896
7,0.125500,0.040812,0.040812,0.202020,0.166364,0.264224,6.479053
8,0.125500,0.075112,0.075112,0.274066,0.220207,-0.354150,8.186391
9,0.048600,0.039238,0.039238,0.198085,0.143481,0.292606,5.543584
10,0.048600,0.031075,0.031075,0.176282,0.134357,0.439762,5.221559


***** Running Evaluation *****
  Num examples = 120
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-120
Configuration saved in ./results/checkpoint-120/config.json
Model weights saved in ./results/checkpoint-120/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 120
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-240
Configuration saved in ./results/checkpoint-240/config.json
Model weights saved in ./results/checkpoint-240/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 120
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-360
Configuration saved in ./results/checkpoint-360/config.json
Model weights saved in ./results/checkpoint-360/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 120
  Batch size = 20
Saving model checkpoint to ./results/checkpoint-480
Configuration saved in ./results/checkpoint-480/config.json
Model weights saved in ./results/checkpoint-480/pytorch_model.bin
***** Ru

peak memory: 6252.04 MiB, increment: 5446.09 MiB
CPU times: user 2h 21min 10s, sys: 43min 26s, total: 3h 4min 36s
Wall time: 3h 15min 18s


In [21]:
df2 = df[3000:].copy()
df3 = df[0:3000].copy()

In [24]:
%%time
%%memit
sbert_trainer, sbert_model = LinearRegression_with_Transformer(model_name = 'bert-base-uncased', 
                                                                 Data = df2['Review Text'].astype(str), 
                                                                 Target = df2.Rating.astype(float), 
                                                                 test_size = 0.66, 
                                                                 max_length = 126, 
                                                                 num_labels = 1, 
                                                                 num_epochs = 5)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/ma

Epoch,Training Loss,Validation Loss,Mse,Rmse,Mae,R2,Accuracy
1,No log,0.507760,0.507760,0.712573,0.491016,0.581568,0.653206
2,No log,0.434158,0.434158,0.658906,0.438609,0.642221,0.662969
3,No log,0.469371,0.469371,0.685106,0.449530,0.613203,0.669477
4,No log,0.440199,0.440199,0.663475,0.432040,0.637242,0.668738
5,0.769400,0.447230,0.447231,0.668753,0.439436,0.631448,0.669107


***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


Logits: [[3.3508034]
 [4.6908293]
 [4.8653245]
 [4.8855553]
 [4.9706054]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]


Saving model checkpoint to ./results/checkpoint-109
Configuration saved in ./results/checkpoint-109/config.json
Model weights saved in ./results/checkpoint-109/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-246] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


...

Logits: [[3.8679595]
 [4.954284 ]
 [5.03974  ]
 [5.0272856]
 [4.743147 ]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]


Saving model checkpoint to ./results/checkpoint-436
Configuration saved in ./results/checkpoint-436/config.json
Model weights saved in ./results/checkpoint-436/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-984] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


Logits: [[3.782437 ]
 [4.971796 ]
 [5.0880117]
 [5.059329 ]
 [4.7667904]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]


Saving model checkpoint to ./results/checkpoint-545
Configuration saved in ./results/checkpoint-545/config.json
Model weights saved in ./results/checkpoint-545/pytorch_model.bin
Deleting older checkpoint [results/checkpoint-1230] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results/checkpoint-327 (score: 0.6694771096812366).
***** Running Evaluation *****
  Num examples = 13521
  Batch size = 20


Logits: [[4.0661874]
 [5.004098 ]
 [5.0948296]
 [5.0855093]
 [4.8016233]]
Labels: [[2.]
 [5.]
 [5.]
 [4.]
 [4.]]
peak memory: 5813.04 MiB, increment: 0.68 MiB
CPU times: user 12min 42s, sys: 15.6 s, total: 12min 57s
Wall time: 12min 47s


In [ ]:
class MakeTorchData(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        item["labels"] = float(item["labels"])
        return item

    def __len__(self):
        return len(self.labels)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
test_embeddings = tokenizer(df3['Review Text'].astype(str).tolist(), truncation=True, padding=True, max_length=126)
test_dataset = MakeTorchData(test_embeddings, df3.Rating.astype(float))

In [36]:
sbert_trainer.eval_dataset = test_dataset
sbert_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3000
  Batch size = 20


Logits: [[5.065415 ]
 [5.135058 ]
 [3.0691428]
 [5.123178 ]
 [5.1684527]]
Labels: [[4.]
 [5.]
 [3.]
 [5.]
 [5.]]


{'epoch': 5.0,
 'eval_accuracy': 0.6603333333333333,
 'eval_loss': 0.48832669854164124,
 'eval_mae': 0.45958763360977173,
 'eval_mse': 0.4883267283439636,
 'eval_r2': 0.6103774787626888,
 'eval_rmse': 0.6988037824630737,
 'eval_runtime': 13.5094,
 'eval_samples_per_second': 222.068,
 'eval_steps_per_second': 11.103}